### Import Modules

In [1]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

C:\Users\anono\Anaconda2\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [24]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Specify fit parameters

In [25]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 10, # max 10
        'exclude' : [], # put LECs here
        'include_log' : True, # alphaS
        'include_log2' : True
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'ma', #'ma', 'ma-ratio, 'xpt', or 'xpt-ratio'; also 'simultaneous', 'ma-old'
    'F2' : 'FpiFpi', #'FKFpi', 'FpiFpi', 'FKFK'
    'chained' : False,
    'bias_correct' : True,
    
    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_results' : False,
}

In [26]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(fit_type=p_dict['fit_type'], F2=p_dict['F2'], include_FV=(p_dict['order']['vol']>0), 
                                 include_alphaS=p_dict['order']['include_log'], include_logSq=p_dict['order']['include_log2'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'], bias_correct=p_dict['bias_correct'],)

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print bootstrapper
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

D:\Documents\School Work\Research\fk-fpi\priors\ma.csv
Making fits...
Done!
100% complete Time (s):  1.67000007629
Compiling results...

Model: ma_FpiFpi_nnlo_FV_alphaS_logSq

Fitting to nnlo 
 with volume corrections O(10) 
 chained: False 
Fitted/[FLAG] values at physical point (including SU(2) isospin corrections):

F_K / F_pi = 1.1884(31) [1.1932(19)]   (delta_su2 = -0.00156(17))

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
   A_loga 0               0.50 (73)            1               0.0 (1.6)
          1               0.00 (87)        A_a 0               0.70 (51)
      L_5 0           -0.00030 (33)            1               0.00 (87)
          1             0.0000 (50)        A_p 0               1.90 (15)
      A_k 0              -0.3 (1.2)            1               0.0 (1.6)
---
boot0 fit results:
Least Square Fit:
  chi2/dof [dof] = 1.3 [16]    Q = 0.18    logGBF = 63.805



In [37]:
[fit_data[abbr]['aw0'][0] for abbr in fit_data.keys()]

[0.5105,
 0.7036,
 0.7036,
 0.5105,
 0.8804,
 0.5105,
 0.8804,
 0.5105,
 0.7036,
 0.8804,
 0.7036,
 0.7036,
 0.7036,
 0.8804,
 0.8804,
 0.7036,
 0.8804]

In [27]:
print bootstrapper.get_name()

ma_FpiFpi_nnlo_FV_alphaS_logSq


In [22]:
g = bootstrapper.get_fit_parameters()
gv.evalcorr([g[key] for key in 'L_5', 'A_k', 'A_p','A_a', 'A_loga'])


array([[ 1.        , -0.99800218, -0.0545299 ,  0.1228446 ,  0.0925843 ],
       [-0.99800218,  1.        ,  0.00992002, -0.15141994, -0.08118303],
       [-0.0545299 ,  0.00992002,  1.        ,  0.03082725,  0.00487372],
       [ 0.1228446 , -0.15141994,  0.03082725,  1.        , -0.88216676],
       [ 0.0925843 , -0.08118303,  0.00487372, -0.88216676,  1.        ]])

In [34]:
g = bootstrapper.get_fit_parameters()
name = bootstrapper.get_name()


def pickle_fit_parameters(self, g, name):
    filename = self.project_path+'/pickles/'+name+'.p'
    gv.dump(g, filename)
    
    
pickle_fit_parameters(data_loader, g, name)

In [33]:
def unpickle_fit_parameters(self, name):
    filename = self.project_path+'/pickles/'+name+'.p'
    return gv.load(filename)

g = unpickle_fit_parameters(data_loader, name)
gv.evalcorr([g[key] for key in 'L_5', 'A_k', 'A_p','A_a', 'A_loga'])

array([[ 1.        , -0.99800218, -0.0545299 ,  0.1228446 ,  0.0925843 ],
       [-0.99800218,  1.        ,  0.00992002, -0.15141994, -0.08118303],
       [-0.0545299 ,  0.00992002,  1.        ,  0.03082725,  0.00487372],
       [ 0.1228446 , -0.15141994,  0.03082725,  1.        , -0.88216676],
       [ 0.0925843 , -0.08118303,  0.00487372, -0.88216676,  1.        ]])

In [15]:
gv.evalcorr([bootstrapper.fits[0].p[key] for key in 'L_5', 'A_k', 'A_p','A_a', 'A_loga'])

array([[ 1.        , -0.99800218, -0.0545299 ,  0.1228446 ,  0.0925843 ],
       [-0.99800218,  1.        ,  0.00992002, -0.15141994, -0.08118303],
       [-0.0545299 ,  0.00992002,  1.        ,  0.03082725,  0.00487372],
       [ 0.1228446 , -0.15141994,  0.03082725,  1.        , -0.88216676],
       [ 0.0925843 , -0.08118303,  0.00487372, -0.88216676,  1.        ]])